In [109]:
%gui qt

import os
import math
import numpy as np
import pyvista as pv
from OCC.Core.STEPControl import STEPControl_Reader
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopAbs import TopAbs_SOLID, TopAbs_EDGE, TopAbs_WIRE, TopAbs_VERTEX
from OCC.Core.BRep import BRep_Tool
from OCC.Core.BRepMesh import BRepMesh_IncrementalMesh
from OCC.Core.Geom import Geom_Plane
from OCC.Extend.DataExchange import read_step_file
from OCC.Extend.ShapeFactory import get_oriented_boundingbox
from OCC.Display.WebGl.jupyter_renderer import JupyterRenderer
from OCC.Core.BRepAlgoAPI   import BRepAlgoAPI_Section
from OCC.Core.gp            import gp_Pln, gp_Pnt, gp_Dir, gp_Ax2
from OCC.Core.TopoDS        import topods
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeWire, BRepBuilderAPI_MakeFace
from OCC.Core.GProp         import GProp_GProps
from OCC.Core.BRepGProp     import brepgprop
from OCC.Core.AIS import AIS_Trihedron

pv.set_jupyter_backend('trame')  # Try interactive 3D again

!pip install reportlab

In [110]:
# Load the STEP and extract 'solid'
from OCC.Extend.DataExchange import read_step_file
# shape = read_step_file("0444-1 ANGLED.step")
# shape = read_step_file("ncTest.step")
# shape = read_step_file("TestEA.step")
shape = read_step_file("TestUEA.step")
# shape = read_step_file("TestPFC.step")
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopAbs import TopAbs_SOLID
exp = TopExp_Explorer(shape, TopAbs_SOLID)
solid = exp.Current()

In [111]:
import json

def classify_profile(cs, json_path, tol_dim=1.0, tol_area=0.05):
    # cs already contains cs['span_flange'], cs['span_web'], cs['area'], cs['length']
    with open(json_path) as f:
        lib = json.load(f)
    best, best_score = None, float('inf')
    for cat, ents in lib.items():
        for name, info in ents.items():
            dh = abs(cs['span_web'] - info['height'])
            dw = abs(cs['span_flange']    - info['width'])
            if dh > tol_dim or dw > tol_dim: 
                continue
            ea = abs(cs['area'] - info['csa'])/info['csa']
            if ea > tol_area: 
                continue
            el = abs(cs['length'] - info.get('length', cs['length']))/info.get('length', cs['length'])
            score = dh + dw + ea*100 + el*100
            if score < best_score:
                best_score = score
                best = {
                  **info,
                  "Designation":     name,
                  "Category":        cat,
                  "Measured_height": cs['span_web'],
                  "Measured_width":  cs['span_flange'],
                  "Measured_area":   cs['area'],
                  "Measured_length": cs['length'],
                  "Match_score":     score,
                  "Profile_type":    cs.get("profile_type", "Unknown")
                }    
    return best

# cs = compute_section_and_length_and_origin(solid)
# profile = classify_profile(cs, "Shape_classifier_info.json")


In [112]:
import numpy as np
from OCC.Core.BRepAdaptor import BRepAdaptor_Surface
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopAbs import TopAbs_FACE
from OCC.Core.TopoDS import topods
from OCC.Core.GProp import GProp_GProps
from OCC.Core.BRepGProp import brepgprop
from OCC.Core.GeomAbs import GeomAbs_Plane

def fingerprint_shape(solid, obb_dims):
    """
    Analyze face normals and areas to guess profile type: Beam, Channel, Angle, Unknown
    """
    face_areas = []
    face_normals = []

    exp = TopExp_Explorer(solid, TopAbs_FACE)
    while exp.More():
        face = topods.Face(exp.Current())
        surf = BRepAdaptor_Surface(face)
        if surf.GetType() != GeomAbs_Plane:
            exp.Next()
            continue

        # Normal
        n = surf.Plane().Axis().Direction()
        n = np.array([n.X(), n.Y(), n.Z()], dtype=float)
        n /= np.linalg.norm(n)

        # Area
        gp = GProp_GProps()
        brepgprop.SurfaceProperties(face, gp)
        area = gp.Mass()

        face_normals.append(n)
        face_areas.append(area)
        exp.Next()

    # Sort OBB dimensions
    dims = sorted(obb_dims)  # [flange_thickness, web_depth, length]
    thickness, height, length = dims
    slenderness = length / height if height else 0
    aspect_ratio = height / thickness if thickness else 0

    # Area stats
    largest = max(face_areas) if face_areas else 0
    large_faces = [a for a in face_areas if a > 0.8 * largest]

    print("\n🔎 Shape fingerprint debug:")
    print(f"  OBB dimensions (sorted): {np.round(dims,1)}")
    print(f"  Slenderness ratio (L/H): {slenderness:.2f}")
    print(f"  Aspect ratio (H/t):      {aspect_ratio:.2f}")
    print(f"  Total planar faces:      {len(face_areas)}")
    print(f"  Large planar faces:      {len(large_faces)}")
    print(f"  Max area:                {max(face_areas):.1f}" if face_areas else "  Max area: —")
    print(f"  Large face areas:        {[round(a,1) for a in large_faces]}")
    
    # Fingerprint logic
    # --- Check for Beam ---
    if len(face_areas) >= 10 and slenderness > 4 and max(face_areas) > 100000:
        return "I"
    
    # --- Check for Angles ---
    elif len(large_faces) == 2 and slenderness > 6:
        a1, a2 = large_faces
        area_ratio = max(a1, a2) / min(a1, a2)
        if area_ratio < 1.1:
            return "L"
        else:
            return "L"

    # --- Check for Channel ---
    elif len(face_areas) >= 6 and slenderness > 6 and max(face_areas) > 50000:
        return "U"
        
    # --- Fallback ---
    return "Unknown"


In [113]:
def origin_and_axes_for_I(centroid, he, L, F, W):
    origin_v = centroid + he[2]*L + he[1]*W - he[0]*F
    origin_o = centroid + he[2]*L - he[1]*W + he[0]*F
    origin_u = centroid + he[2]*L - he[1]*W - he[0]*F
    face_axes = {
        'V': (-L, F),
        'O': (-L, W),
        'U': (-L, W),
    }
    return origin_v, origin_o, origin_u, face_axes

In [114]:
def origin_and_axes_for_U(centroid, he, L, F, W):
    origin_v = centroid + he[2]*L + he[1]*W - he[0]*F
    origin_o = centroid + he[2]*L - he[1]*W + he[0]*F
    origin_u = centroid + he[2]*L - he[1]*W - he[0]*F
    face_axes = {
        'H': (-L, F),    # back face
        'U': (-L, W),    # web leg
        'O': (-L, W)     # flange leg
    }
    return origin_v, origin_o, origin_u, face_axes

In [115]:
def origin_and_axes_for_L(centroid, he, L, F, W):
    origin_v = centroid + he[2]*L - he[0]*F - he[1]*W
    origin_o = centroid + he[2]*L + he[0]*F - he[1]*W
    origin_u = centroid + he[2]*L - he[0]*F + he[1]*W
    face_axes = {
        'H': (-L, F),   # heel leg
        'U': (-L, W)    # upstand leg
    }
    return origin_v, origin_o, origin_u, face_axes

In [116]:
def get_views_for_profile(profile_type):
    if profile_type == "I":
        return ['V', 'O', 'U']
    elif profile_type == "U":
        return ['H', 'O', 'U']
    elif profile_type == "L":
        return ['H', 'U']
    else:
        return ['H']


In [117]:
def compute_principal_axes_and_extents(solid):
    from OCC.Core.GProp import GProp_GProps
    from OCC.Core.BRepGProp import brepgprop
    from OCC.Core.BRepAlgoAPI import BRepAlgoAPI_Section
    from OCC.Core.TopExp import TopExp_Explorer
    from OCC.Core.TopAbs import TopAbs_EDGE
    from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeWire, BRepBuilderAPI_MakeFace
    from OCC.Core.TopoDS import topods
    from OCC.Core.gp import gp_Pnt, gp_Dir, gp_Pln

    obb_ret = get_oriented_boundingbox(solid)
    center_pnt = obb_ret[0]
    half_extents = np.array(obb_ret[1], dtype=float)

    flange_idx, web_idx, long_idx = np.argsort(half_extents)
    span_flange = 2.0 * half_extents[flange_idx]
    span_web = 2.0 * half_extents[web_idx]
    length = 2.0 * half_extents[long_idx]

    # Axes: L along part, F = flange direction, W = web/upstand direction
    if len(obb_ret) == 4:
        obb_axes = obb_ret[3]
        axes_np = [np.array([d.X(), d.Y(), d.Z()], float) for d in obb_axes]
        L = axes_np[long_idx] / np.linalg.norm(axes_np[long_idx])
        F = axes_np[flange_idx] / np.linalg.norm(axes_np[flange_idx])
        W = axes_np[web_idx] / np.linalg.norm(axes_np[web_idx])
    else:
        props = GProp_GProps(); brepgprop.VolumeProperties(solid, props)
        im = props.MatrixOfInertia()
        M = np.array([[im.Value(i,j) for j in (1,2,3)] for i in (1,2,3)])
        eigvals, eigvecs = np.linalg.eigh(M)
        order = np.argsort(eigvals)
        L, F, W = eigvecs[:,order[0]], eigvecs[:,order[1]], eigvecs[:,order[2]]
        L, F, W = L/np.linalg.norm(L), F/np.linalg.norm(F), W/np.linalg.norm(W)

    if hasattr(center_pnt, 'X'):
        center = gp_Pnt(center_pnt.X(), center_pnt.Y(), center_pnt.Z())
    else:
        center = gp_Pnt(*center_pnt)

    # Cross-section area from section cut
    plane = gp_Pln(center, gp_Dir(*L))
    sec = BRepAlgoAPI_Section(solid, plane)
    sec.ComputePCurveOn1(True); sec.Approximation(True); sec.Build()
    exp_e = TopExp_Explorer(sec.Shape(), TopAbs_EDGE)
    wb = BRepBuilderAPI_MakeWire()
    while exp_e.More():
        wb.Add(topods.Edge(exp_e.Current()))
        exp_e.Next()
    face = BRepBuilderAPI_MakeFace(wb.Wire()).Face()
    gp2 = GProp_GProps(); brepgprop.SurfaceProperties(face, gp2)
    area = gp2.Mass()

    centroid = np.array([center.X(), center.Y(), center.Z()], dtype=float)

    return {
        'axes': (L, F, W),
        'half_extents': half_extents,
        'flange_idx': flange_idx,
        'web_idx': web_idx,
        'long_idx': long_idx,
        'span_flange': span_flange,
        'span_web': span_web,
        'length': length,
        'area': area,
        'centroid': centroid,
        'plane': plane
    }


In [118]:
def origin_and_axes_for_I(geo):
    L, F, W = geo['axes']
    he = geo['half_extents']
    iL, iF, iW = geo['long_idx'], geo['flange_idx'], geo['web_idx']
    c = geo['centroid']

    return {
        'origin_v': c + he[iL]*L + he[iF]*W - he[iW]*F,
        'origin_o': c + he[iL]*L - he[iF]*W + he[iW]*F,
        'origin_u': c + he[iL]*L - he[iF]*W - he[iW]*F,
        'face_axes': {
            'V': (-L, F),
            'O': (-L, W),
            'U': (-L, W),
        }
    }


In [119]:
def origin_and_axes_for_U(geo):
    L, F, W = geo['axes']
    he = geo['half_extents']
    iL, iF, iW = geo['long_idx'], geo['flange_idx'], geo['web_idx']
    c = geo['centroid']

    return {
        'origin_h': c + he[iL]*L + he[iF]*F - he[iW]*W,
        'origin_o': c + he[iL]*L + he[iF]*F + he[iW]*W,
        'origin_u': c + he[iL]*L - he[iF]*F + he[iW]*W,
        'face_axes': {
            'H': (-L, F),
            'O': (-L, W),
            'U': (-L, W),
        }
    }


In [120]:
def origin_and_axes_for_L(geo):
    L, F, W = geo['axes']
    he = geo['half_extents']
    iL, iF, iW = geo['long_idx'], geo['flange_idx'], geo['web_idx']
    c = geo['centroid']

    return {
        'origin_h': c + he[iL]*L + he[iF]*F - he[iW]*W,
        'origin_u': c + he[iL]*L - he[iF]*F + he[iW]*W,
        'face_axes': {
            'H': (-L, F),
            'U': (-L, W),
        }
    }


In [121]:
def get_cs_for_shape(solid):
    """
    Analyze solid geometry and return cs dict with:
      - origin_* points
      - axes (L, F, W)
      - span_flange, span_web, length
      - face_axes mapping
    Automatically routes based on shape fingerprint (I, U, L).
    """
    geo = compute_principal_axes_and_extents(solid)
    obb_dims = geo['half_extents']
    shape_type = fingerprint_shape(solid, obb_dims)

    if shape_type == 'I':
        origins = origin_and_axes_for_I(geo)
    elif shape_type == 'U':
        origins = origin_and_axes_for_U(geo)
    elif shape_type == 'L':
        origins = origin_and_axes_for_L(geo)
    else:
        raise ValueError(f"Unsupported or unidentified profile shape: {shape_type}")

    return {**geo, **origins, 'profile_type': shape_type}


In [122]:
cs = get_cs_for_shape(solid)
print(f"\n🧠 Detected shape type: {cs['profile_type']} \n")
print(f"cs: {cs}")
profile = classify_profile(cs, "Shape_classifier_info.json")
print(f"Profile: {profile}")


🔎 Shape fingerprint debug:
  OBB dimensions (sorted): [ 24.5  54.7 500. ]
  Slenderness ratio (L/H): 9.14
  Aspect ratio (H/t):      2.23
  Total planar faces:      8
  Large planar faces:      2
  Max area:                99921.5
  Large face areas:        [99921.5, 81921.5]

🧠 Detected shape type: L 

cs: {'axes': (array([ 1.85964943e-05, -4.52739153e-05,  9.99999999e-01]), array([-2.52857260e-01,  9.67503594e-01,  4.85049344e-05]), array([-9.67503595e-01, -2.52857261e-01,  6.54433687e-06])), 'half_extents': array([ 24.5087121 ,  54.72086365, 500.00272801]), 'flange_idx': 0, 'web_idx': 1, 'long_idx': 2, 'span_flange': 49.01742420924849, 'span_web': 109.4417273083333, 'length': 1000.0054560184665, 'area': 870.8672600886503, 'centroid': array([  2.57550564,  11.78353098, 500.00008066]), 'plane': <class 'gp_Pln'>, 'origin_h': array([  49.33023042,   49.30972863, 1000.00363875]), 'origin_u': array([ -44.16062255,  -25.78794084, 1000.00197739]), 'face_axes': {'H': (array([-1.85964943e-05

In [123]:
from OCC.Display.SimpleGui import init_display
from OCC.Core.gp import gp_Pnt, gp_Dir
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeEdge
import numpy as np

# === Init display
display, start_display, add_menu, add_function_to_menu = init_display()

# === Display solid
display.DisplayShape(solid, update=True)

# === Helper: draw an axis arrow from origin in a direction
def draw_axis_arrow(origin, direction, scale=100, color='BLACK'):
    end = gp_Pnt(
        origin.X() + direction.X() * scale,
        origin.Y() + direction.Y() * scale,
        origin.Z() + direction.Z() * scale,
    )
    edge = BRepBuilderAPI_MakeEdge(origin, end).Edge()
    display.DisplayShape(edge, color=color, update=False)

# === Helper: draw OBB from center, half_extents and axes
def draw_obb(display, center, half_extents, axes, color="YELLOW"):
    if hasattr(center, 'X'):
        c = np.array([center.X(), center.Y(), center.Z()], dtype=float)
    else:
        c = np.array(center, dtype=float)

    axis_vecs = [
        np.array([ax.X(), ax.Y(), ax.Z()], float) if hasattr(ax, 'X') else np.array(ax)
        for ax in axes
    ]

    # Generate 8 corner points
    corners = []
    for dx in (-1, 1):
        for dy in (-1, 1):
            for dz in (-1, 1):
                offset = (
                    dx * half_extents[0] * axis_vecs[0] +
                    dy * half_extents[1] * axis_vecs[1] +
                    dz * half_extents[2] * axis_vecs[2]
                )
                pt = c + offset
                corners.append(gp_Pnt(*pt))

    # Define box edges by connecting corners
    edges = [
        (0, 1), (0, 2), (0, 4),
        (1, 3), (1, 5),
        (2, 3), (2, 6),
        (3, 7),
        (4, 5), (4, 6),
        (5, 7),
        (6, 7)
    ]
    for i, j in edges:
        edge = BRepBuilderAPI_MakeEdge(corners[i], corners[j]).Edge()
        display.DisplayShape(edge, color=color, update=False)

# === Draw L, F, W axes from centroid
origin = gp_Pnt(*cs['centroid'])
L, F, W = cs['axes']

draw_axis_arrow(origin, gp_Dir(*L), scale=100, color='BLUE')   # L axis
draw_axis_arrow(origin, gp_Dir(*F), scale=100, color='GREEN')  # F axis
draw_axis_arrow(origin, gp_Dir(*W), scale=100, color='RED')    # W axis

# === Draw the OBB
draw_obb(display, cs['centroid'], cs['half_extents'], cs['axes'], color="YELLOW")

# === Launch viewer
display.FitAll()
start_display()


pyside6 backend - Qt version 6.8.3
Many colors for color name BLUE, using first.


In [30]:
import pyvista as pv
import numpy as np

# Replace with actual values from your cs
cs = {
    'centroid': np.array([2.58, 11.78, 500.0]),
    'origin_h': np.array([49.33, 49.31, 1000.0]),
    'origin_u': np.array([-44.16, -25.79, 1000.0]),
    'axes': (
        np.array([1.86e-5, -4.53e-5, 1.0]),     # L
        np.array([-0.253, 0.968, 4.85e-5]),     # F
        np.array([-0.968, -0.253, 6.54e-6])     # W
    )
}

# Optional: your mesh from STEP file using OCC or other source
# Here we simulate a 1000mm L, 100mm H, 50mm W solid
mesh = pv.Cube(center=(0, 0, 500), x_length=50, y_length=100, z_length=1000)

plotter = pv.Plotter()
plotter.add_mesh(mesh, opacity=0.3, show_edges=True, color="lightgray")

# Show centroid and origins
plotter.add_mesh(pv.Sphere(radius=5.0, center=cs['centroid']), color="blue", name="centroid")
plotter.add_mesh(pv.Sphere(radius=4.0, center=cs['origin_h']), color="green", name="origin_h")
plotter.add_mesh(pv.Sphere(radius=4.0, center=cs['origin_u']), color="red", name="origin_u")

# Axes from centroid
# Axes from centroid
L, F, W = cs['axes']
origin = cs['centroid']
scale = 100

# Draw L axis (black)
arrow_L = pv.Arrow(start=origin, direction=L * scale, tip_length=10)
plotter.add_mesh(arrow_L, color='black', name='L_axis')

# Draw F axis (green)
arrow_F = pv.Arrow(start=origin, direction=F * scale, tip_length=10)
plotter.add_mesh(arrow_F, color='green', name='F_axis')

# Draw W axis (orange)
arrow_W = pv.Arrow(start=origin, direction=W * scale, tip_length=10)
plotter.add_mesh(arrow_W, color='orange', name='W_axis')

plotter.show_grid()
plotter.view_isometric()
plotter.show()


Widget(value='<iframe src="http://localhost:11549/index.html?ui=P_0x122b543a450_1&reconnect=auto" class="pyvis…

In [31]:
import ipywidgets as widgets
widgets.IntSlider()


IntSlider(value=0)

In [18]:
# Prep Header data

# Store for DSTV header
header_dict = {
    "Designation":       profile['Designation'],
    "Mass":              profile['mass'],
    "Height":            profile['height'],
    "Width":             profile['width'],
    "CSA":               profile['csa'],
    "web_thickness":     profile['web_thickness'],
    "flange_thickness":  profile['flange_thickness'],
    "root_radius":       profile['root_radius'],
    "toe_radius":        profile['toe_radius'],
    "code_profile":      profile['code_profile'],
    "Length":            profile['Measured_length']
}

> Colour Faces

In [19]:
import numpy as np
from OCC.Core.TopExp         import TopExp_Explorer
from OCC.Core.TopAbs         import TopAbs_FACE
from OCC.Core.TopoDS         import topods
from OCC.Core.BRepAdaptor    import BRepAdaptor_Surface
from OCC.Core.GeomAbs        import GeomAbs_Plane
from OCC.Core.GProp          import GProp_GProps
from OCC.Core.BRepGProp      import brepgprop

def find_VOU_faces(solid, cs, tol=0.85, area_frac=0.7):
    """
    Identify and return the three principal planar faces:
      - V: web face (red)
      - O: outside flange face (green)
      - U: underside flange face (blue)
    based on axis alignment (cs['axes']) and area thresholds.
    Returns a dict: {'V': face, 'O': face, 'U': face}
    """
    L, F, W    = cs['axes']
    centroid   = cs['centroid']
    span_web   = cs['span_web']
    span_fl    = cs['span_flange']
    length     = cs['length']
    # approximate full face areas
    area_web   = span_web    * length
    area_fl    = span_fl     * length
    thr_web    = area_web    * area_frac
    thr_fl     = area_fl     * area_frac

    faces = {'V': None, 'O': None, 'U': None}

    exp = TopExp_Explorer(solid, TopAbs_FACE)
    while exp.More():
        f   = topods.Face(exp.Current())
        sad = BRepAdaptor_Surface(f, True)
        if sad.GetType() != GeomAbs_Plane:
            exp.Next()
            continue

        # Face normal
        pln = sad.Plane()
        nv  = pln.Axis().Direction().XYZ()
        n   = np.array([nv.X(), nv.Y(), nv.Z()], float)

        # Face area
        gp  = GProp_GProps(); brepgprop.SurfaceProperties(f, gp)
        area = gp.Mass()

        # Classify Web face (V)
        # normal ∥ W and large area, pick the one with positive centroid·W
        if faces['V'] is None and abs(np.dot(n, W)) > tol and area > thr_web:
            # ensure it's the 'outer' web side
            cprops = GProp_GProps(); brepgprop.SurfaceProperties(f, cprops)
            cc = cprops.CentreOfMass()
            fc = np.array([cc.X(), cc.Y(), cc.Z()], float)
            if (fc - centroid).dot(W) > 0:
                faces['V'] = f

        # Classify flange faces (O/U)
        if abs(np.dot(n, F)) > tol and area > thr_fl:
            # Outside flange (O): normal·F > 0
            if faces['O'] is None and np.dot(n, F) > 0:
                faces['O'] = f
            # Underside flange (U): normal·F < 0
            if faces['U'] is None and np.dot(n, F) < 0:
                faces['U'] = f
        
        exp.Next()

    return faces


In [20]:
# Example usage:
vou_faces = find_VOU_faces(solid, cs)
for key, val in vou_faces.items():
    print(f"{key} -> {val} ({type(val)})")


V -> <class 'TopoDS_Face'> (<class 'OCC.Core.TopoDS.TopoDS_Face'>)
O -> <class 'TopoDS_Face'> (<class 'OCC.Core.TopoDS.TopoDS_Face'>)
U -> <class 'TopoDS_Face'> (<class 'OCC.Core.TopoDS.TopoDS_Face'>)


In [21]:
import numpy as np
from OCC.Core.GProp import GProp_GProps
from OCC.Core.BRepGProp import brepgprop
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopAbs import TopAbs_FACE
from OCC.Core.TopoDS import topods
from OCC.Core.GeomAbs import GeomAbs_Plane
from OCC.Core.Quantity import Quantity_Color, Quantity_TOC_RGB
from OCC.Core.gp import gp_Pnt
from OCC.Core.BRepPrimAPI import BRepPrimAPI_MakeSphere
from OCC.Core.BRepBuilderAPI  import BRepBuilderAPI_MakeEdge
from OCC.Display.SimpleGui import init_display

# Assume compute_section_and_length_and_origin and find_VOU_faces are already defined and imported
# Compute the section/origin/axes info
# cs = compute_section_and_length_and_origin(solid)
vou = find_VOU_faces(solid, cs)

# Extract faces and origins
V_face, O_face, U_face = vou['V'], vou['O'], vou['U']
origins = {
    'V': np.array(cs['origin_v'], float),
    'O': np.array(cs['origin_o'], float),
    'U': np.array(cs['origin_u'], float),
}

#   face_axes: {'V':(axis_x,axis_y), 'O':…, 'U':…}
face_axes = {k: (np.array(ax), np.array(ay)) for k,(ax,ay) in cs['face_axes'].items()}
axis_len = cs['length'] * 0.05  # 5% of total length

# Set up Qt5 display
display, start_display, _, _ = init_display()
grey = Quantity_Color(0.8, 0.8, 0.8, Quantity_TOC_RGB)
display.DisplayShape(solid, color=grey, transparency=0.5, update=False)

# Color the V, O, U faces
colV = Quantity_Color(1.0, 0.0, 0.0, Quantity_TOC_RGB)  # red
colO = Quantity_Color(0.0, 1.0, 0.0, Quantity_TOC_RGB)  # green
colU = Quantity_Color(0.0, 0.0, 1.0, Quantity_TOC_RGB)  # blue

if V_face:
    display.DisplayShape(V_face, color=colV, update=False)
if O_face:
    display.DisplayShape(O_face, color=colO, update=False)
if U_face:
    display.DisplayShape(U_face, color=colU, update=False)

# Draw the origin_v
origin = cs['origin_v']
yellow = Quantity_Color(1.0, 1.0, 0.0, Quantity_TOC_RGB)
sphere = BRepPrimAPI_MakeSphere(gp_Pnt(*origin), 5.0).Shape()
display.DisplayShape(sphere, color=colV, update=False)

# Draw the origin_o
origin = cs['origin_o']
yellow = Quantity_Color(1.0, 1.0, 0.0, Quantity_TOC_RGB)
sphere = BRepPrimAPI_MakeSphere(gp_Pnt(*origin), 5.0).Shape()
display.DisplayShape(sphere, color=colO, update=False)

# Draw the origin_u
origin = cs['origin_u']
yellow = Quantity_Color(1.0, 1.0, 0.0, Quantity_TOC_RGB)
sphere = BRepPrimAPI_MakeSphere(gp_Pnt(*origin), 5.0).Shape()
display.DisplayShape(sphere, color=colU, update=False)

# Draw local axes at each face origin
axis_len = cs['length'] * 0.05  # 5% of total length

for code in ['V','O','U']:
    origin = origins[code]
    ax, ay = face_axes[code]
    p0 = gp_Pnt(*origin.tolist())

    # +X in green
    p1 = gp_Pnt(*(origin + axis_len*ax).tolist())
    shapes_x = display.DisplayShape(
        BRepBuilderAPI_MakeEdge(p0, p1).Edge(),
        color=Quantity_Color(0,1,0,Quantity_TOC_RGB),
        update=False
    )
    # thicken each AIS object returned
    for ais in shapes_x:
        ais.SetWidth(4)

    # +Y in red
    p2 = gp_Pnt(*(origin + axis_len*ay).tolist())
    shapes_y = display.DisplayShape(
        BRepBuilderAPI_MakeEdge(p0, p2).Edge(),
        color=Quantity_Color(1,0,0,Quantity_TOC_RGB),
        update=False
    )
    for ais in shapes_y:
        ais.SetWidth(4) 

# Finalize view
display.FitAll()
display.View_Iso()
# start_display()


KeyError: 'origin_v'

In [10]:
import numpy as np
from OCC.Core.TopAbs      import TopAbs_WIRE
from OCC.Core.TopExp      import TopExp_Explorer
from OCC.Core.TopoDS      import topods
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeFace
from OCC.Core.BRepGProp   import brepgprop
from OCC.Core.GProp       import GProp_GProps
from OCC.Core.BRep        import BRep_Tool


def find_holes_on_faces(vou_faces, section_info):
    """
    vou_faces: dict {'V': TopoDS_Face, 'O':..., 'U':...}
    section_info: output of compute_section_and_length_and_origin, containing:
      'origin_v','origin_o','origin_u', and 'face_axes' mapping codes->(axis_x,axis_y)
    Returns:
      hole_info: {
        'V': [ {'area':..., 'local_xy':(x,y)}, ... ],
        'O': [...],
        'U': [...],
      }
    Coordinates x,y are measured from each face's origin along its face_axes.
    """
    # unpack origins and face_axes
    origins = {
        'V': np.array(section_info['origin_v'], float),
        'O': np.array(section_info['origin_o'], float),
        'U': np.array(section_info['origin_u'], float),
    }
    face_axes = section_info['face_axes']  # {'V':(L,F), 'O':(L,W), 'U':(L,-W)}

    hole_info = {}
    for code, face in vou_faces.items():
        # gather wires on face
        exp = TopExp_Explorer(face, TopAbs_WIRE)
        wires = []
        while exp.More():
            wires.append(topods.Wire(exp.Current()))
            exp.Next()
        # if only outer boundary, no holes
        if len(wires) <= 1:
            hole_info[code] = []
            continue
        # get underlying surface
        surf = BRep_Tool.Surface(face)
        origin = origins[code]
        axis_x, axis_y = face_axes[code]

        holes = []
        # for each inner wire
        for wire in wires[1:]:
            hole_face = BRepBuilderAPI_MakeFace(surf, wire).Face()
            props = GProp_GProps()
            brepgprop.SurfaceProperties(hole_face, props)
            area = props.Mass()
            cog = props.CentreOfMass()
            # global centroid
            gx, gy, gz = float(cog.X()), float(cog.Y()), float(cog.Z())
            global_pt = np.array([gx, gy, gz], float)
            # vector from face origin
            rel = global_pt - origin
            # project into face local coords
            x_loc = float(np.dot(rel, axis_x))
            y_loc = float(np.dot(rel, axis_y))
            holes.append({
                'area': area,
                'local_xy': (x_loc, y_loc)
            })
        hole_info[code] = holes

    return hole_info

# Example usage:
vou_faces = find_VOU_faces(solid, cs)
section_info = compute_section_and_length_and_origin(solid)
hole_info = find_holes_on_faces(vou_faces, section_info)
for code, holes in hole_info.items():
    print(f"Face {code}: {len(holes)} holes")
    for i, h in enumerate(holes,1):
        x,y = h['local_xy']
        print(f"  Hole {i}: area={h['area']:.2f}, x={x:.2f}, y={y:.2f}")



=== OBB Axis Assignment ===
Half-extents: [ 101.6 1140.   101.8]
Sorted indices → web_idx: 2, flange_idx: 0, long_idx: 1
Dimensions: web: 203.6 mm, flange: 203.2 mm, length: 2280.0 mm
Face V: 8 holes
  Hole 1: area=113.10, x=2000.00, y=151.80
  Hole 2: area=113.10, x=1500.00, y=151.80
  Hole 3: area=113.10, x=900.00, y=151.80
  Hole 4: area=113.10, x=300.00, y=151.80
  Hole 5: area=113.10, x=300.00, y=51.80
  Hole 6: area=113.10, x=900.00, y=51.80
  Hole 7: area=113.10, x=1500.00, y=51.80
  Hole 8: area=113.10, x=2000.00, y=51.80
Face O: 13 holes
  Hole 1: area=113.10, x=425.00, y=49.85
  Hole 2: area=113.10, x=2025.00, y=49.74
  Hole 3: area=113.10, x=1225.00, y=49.79
  Hole 4: area=153.94, x=680.00, y=153.43
  Hole 5: area=153.94, x=230.00, y=153.46
  Hole 6: area=153.94, x=280.00, y=153.46
  Hole 7: area=153.94, x=730.00, y=153.43
  Hole 8: area=78.54, x=954.99, y=26.61
  Hole 9: area=78.54, x=1285.01, y=176.59
  Hole 10: area=78.54, x=1930.00, y=153.34
  Hole 11: area=78.54, x=133

In [11]:
# 1) After you’ve computed cs and vou_faces:
vou_faces = find_VOU_faces(solid, cs)
hole_info = find_holes_on_faces(vou_faces, cs)

# 2) Now overlay the hole spheres in the same display
from OCC.Core.gp           import gp_Pnt
from OCC.Core.Quantity     import Quantity_Color, Quantity_TOC_RGB
from OCC.Core.BRepPrimAPI  import BRepPrimAPI_MakeSphere

# small radius for check spheres
r = min(cs['span_flange'], cs['span_web']) * 0.01

for code, holes in hole_info.items():
    origin = origins[code]
    axis_x, axis_y = face_axes[code]
    for h in holes:
        x_loc, y_loc = h['local_xy']
        pt = origin + x_loc*axis_x + y_loc*axis_y
        p  = gp_Pnt(*pt.tolist())
        sph = BRepPrimAPI_MakeSphere(p, r).Shape()
        col = {'V':(1,0,0), 'O':(0,1,0), 'U':(0,0,1)}[code]
        display.DisplayShape(sph,
                            color=Quantity_Color(*col,Quantity_TOC_RGB),
                            transparency=0.6,
                            update=False)

display.FitAll()
display.Repaint()
# start_display()


In [12]:
import numpy as np
import pandas as pd
import math

# Assume hole_info and cs (from compute_section_and_length_and_origin) are available
# cs['face_axes'] gives a dict: {'V':(axis_x,axis_y),'O':(axis_x,axis_y),'U':(axis_x,axis_y)}
# hole_info: dict mapping face codes to lists of {'area':…, 'local_xy':(x,y)}

# 1) Unpack face‐axes
face_axes = {
    code: (np.array(ax, float), np.array(ay, float))
    for code, (ax, ay) in cs['face_axes'].items()
}

# 2) Build hole DataFrame directly from local coordinates
rows = []
for face_code, holes in hole_info.items():
    axis_x, axis_y = face_axes[face_code]
    for idx, h in enumerate(holes, start=1):
        x_loc, y_loc = h['local_xy']
        area_mm2 = h['area']
        diam_mm  = 2.0 * math.sqrt(area_mm2 / math.pi)
        rows.append({
            'Hole #':        idx,
            'Code':          face_code,
            'Diameter (mm)': diam_mm,
            'X (mm)':        x_loc,
            'Y (mm)':        y_loc,
        })

# 3) Create DataFrame
df_holes = pd.DataFrame(rows,
                        columns=['Hole #','Code','Diameter (mm)','X (mm)','Y (mm)'])
print(df_holes)
# df_holes.to_csv("holes.csv", index=False)


    Hole # Code  Diameter (mm)       X (mm)      Y (mm)
0        1    V           12.0  2000.007481  151.880968
1        2    V           12.0  1500.007482  151.880968
2        3    V           12.0   900.007484  151.880968
3        4    V           12.0   300.007485  151.880968
4        5    V           12.0   300.007485   51.880968
5        6    V           12.0   900.007484   51.880968
6        7    V           12.0  1500.007482   51.880968
7        8    V           12.0  2000.007481   51.880968
8        1    O           12.0   425.003550   49.850783
9        2    O           12.0  2025.003546   49.737144
10       3    O           12.0  1225.003548   49.793963
11       4    O           14.0   680.010908  153.432671
12       5    O           14.0   230.010909  153.464632
13       6    O           14.0   280.010909  153.461081
14       7    O           14.0   730.010908  153.429120
15       8    O           10.0   955.001901   26.613140
16       9    O           10.0  1285.012554  176

In [13]:
# Check for duplicate locations on same face

import math
import pandas as pd
import numpy as np

from OCC.Core.BRepPrimAPI  import BRepPrimAPI_MakeSphere
from OCC.Core.gp           import gp_Pnt
from OCC.Core.Quantity     import Quantity_Color, Quantity_TOC_RGB

# Assumes the following are already in scope:
#   df             : pandas.DataFrame with columns ['Hole #','Code','Diameter (mm)','X (mm)','Y (mm)']
#   hole_info      : dict mapping face codes to lists of hole dicts with 'area' and 'centroid'
#   cs   : dict from compute_section_and_length_and_origin(solid)
#   solid          : the loaded OCC solid shape

# 1) Check for duplicate X/Y within each Code group
print("Checking for duplicate hole positions within each face code...")
dups = df_holes[df_holes.duplicated(subset=['Code','X (mm)','Y (mm)'], keep=False)]
if not dups.empty:
    print("Duplicate hole positions found:")
    print(dups)
else:
    print("No duplicate hole positions detected.")

Checking for duplicate hole positions within each face code...
No duplicate hole positions detected.


In [14]:
# %%
"""
Jupyter Notebook: Write DSTV NC1 ST-block header to .nc1 file with specified order
"""
# Cell 1: Define DSTV header fields inline
# Replace placeholder values with your actual data
project_number     = 'PROJ-123'
out_filename        = '0444-1'
model_filename     = 'model.step'
material_grade     = 'S355JR'
quantity           = 1

faces = ['V', 'U', 'O']  #Face priority order

# %%
# Cell 2: Write the DSTV NC1 ST-block header to '0444-1.nc1'
filename = f"{out_filename}.nc1"
with open(filename, 'w') as f:
    f.write('ST\n')
    f.write(f"  Project Number\n")
    f.write(f"  Drawing Number\n")
    f.write(f"  {out_filename}\n")
    f.write(f"  {model_filename}\n")
    f.write(f"  {material_grade}\n")
    f.write(f"  {quantity}\n")
    f.write(f"  {header_dict['Designation']}\n")
    f.write(f"  {header_dict['code_profile']}\n")
    f.write(f"    {header_dict['Length']:8.2f}\n")
    f.write(f"    {header_dict['Height']:8.2f}\n")
    f.write(f"    {header_dict['Width']:8.2f}\n")
    f.write(f"    {header_dict['flange_thickness']:8.2f}\n")
    f.write(f"    {header_dict['web_thickness']:8.2f}\n")
    f.write(f"    {header_dict['root_radius']:8.2f}\n")
    f.write(f"    {header_dict['Mass']:8.2f}\n")
    f.write('        0.00\n') #surface area
    # Following the spec, three zeros
    f.write('        0.00\n')
    f.write('        0.00\n')
    f.write('        0.00\n')
    f.write('        0.00\n')
    f.write('  -\n')
    f.write('  -\n')
    f.write('  -\n')
    f.write('  -\n')
    
    # BO blocks by face
    for face in faces:
        df_face = df_holes[df_holes['Code'] == face]
        if df_face.empty:
            continue
        f.write('BO\n')
        for _, row in df_face.iterrows():
            x = row['X (mm)']
            y = row['Y (mm)']
            d = row['Diameter (mm)']
            # Right-align numeric columns: x, y (8-wide), diameter (6-wide)
            f.write(f"  {face.lower()}  {x:8.2f}u {y:8.2f} {d:6.2f}\n")
            
    f.write('EN\n')
print(f"DSTV header written to {filename}")

DSTV header written to 0444-1.nc1


In [15]:
# — assume you’ve already run:
# cs          = compute_section_and_length_and_origin(solid)
# vou_faces   = find_VOU_faces(solid, cs)
# hole_info   = find_holes_on_faces(vou_faces, cs)
# display, ... = init_display(); display.DisplayShape(solid, ...)

# And that you already drew your axes from cs['face_axes']…

# Now simply project your df_holes using the same source of truth:

# sphere colors
color_map = {
    'V': Quantity_Color(1,0,0,Quantity_TOC_RGB),
    'O': Quantity_Color(0,1,0,Quantity_TOC_RGB),
    'U': Quantity_Color(0,0,1,Quantity_TOC_RGB),
}

# Reuse the origins and face_axes from cs
origins  = {
    'V': np.array(cs['origin_v'], float),
    'O': np.array(cs['origin_o'], float),
    'U': np.array(cs['origin_u'], float),
}
axis_map = { face: (np.array(ax), np.array(ay))
             for face,(ax,ay) in cs['face_axes'].items() }

# Plot each sphere
for row in df_holes.itertuples(index=False):
    _, code, diam_mm, x_loc, y_loc = row
    # directly use the single source of truth:
    origin  = origins[code]
    axis_x, axis_y = axis_map[code]

    pt = origin + x_loc*axis_x + y_loc*axis_y
    center = gp_Pnt(*pt.tolist())
    sph = BRepPrimAPI_MakeSphere(center, diam_mm/2.0).Shape()
    display.DisplayShape(sph,
                        color=color_map[code],
                        update=False)

display.FitAll()
display.Repaint()


In [16]:
import math
import numpy as np
import pandas as pd
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4, portrait
from reportlab.lib.units import mm

# === User-provided context ===
# hole_info: dict mapping 'V','O','U' to lists of {'area':float,'local_xy':(x,y)}
# cs: dict from compute_section_and_length_and_origin(solid), containing:
#     - 'origin_v','origin_o','origin_u': 3-element arrays
#     - 'face_axes': {'V':(ax,ay), 'O':(ax,ay), 'U':(ax,ay)}
#     - 'length', 'span_flange', 'span_web'
#     - df_holes: DataFrame with flattened hole data table

def round_nearest_0_5(x):
    return round(x * 2) / 2

# Rebuild a flat DataFrame from hole_info
rows = []
for face, holes in hole_info.items():
    for idx, h in enumerate(holes, start=1):
        x, y = h['local_xy']
        d = 2.0 * math.sqrt(h['area'] / math.pi)
        rows.append({'Hole #': idx,
                     'Code': face,
                     'X (mm)': round_nearest_0_5(abs(x)),
                     'Y (mm)': round_nearest_0_5(abs(y)),
                     'Diameter (mm)': d})
df = pd.DataFrame(rows, columns=['Hole #','Code','X (mm)','Y (mm)','Diameter (mm)'])

PAGE_SIZE = portrait(A4)
PAGE_W, PAGE_H = PAGE_SIZE
MARGIN    = 15 * mm
TITLE_H   = 12 * mm
panel_w   = PAGE_W - 2 * MARGIN
panel_h   = 0.25 * PAGE_H

c = canvas.Canvas("hole_drawing_views.pdf", pagesize=PAGE_SIZE)
c.setLineWidth(1.2)
c.rect(MARGIN, MARGIN, PAGE_W-2*MARGIN, PAGE_H-2*MARGIN)

views = [('O', "Outside Flange (O) View"), ('V', "Vertical Flange (V) View"), ('U', "Upper Flange (U) View")]

for i, (code, title) in enumerate(views):
    w_mm = cs['length']
    h_mm = cs['span_flange'] if code != 'V' else cs['span_web']

    scale_x = (panel_w - 20 * mm) / w_mm if w_mm else 1
    scale_y = (panel_h - 50 * mm) / h_mm if h_mm else 1
    scale = min(scale_x, scale_y)

    # Panel origin (top down stack)
    ox = MARGIN + 10 * mm
    oy = PAGE_H - MARGIN - TITLE_H - i * panel_h - panel_h

    # Title
    c.setFont("Helvetica-Bold", 12)
    c.drawCentredString(PAGE_W/2, oy + panel_h - 25 * mm, f"Hole Layout - {title}")

    # Axes
    c.setLineWidth(0.6)
    c.line(ox, oy, ox + panel_w - 25 * mm, oy)
    c.line(ox, oy, ox, oy + panel_h - 50 * mm)
    c.setFont("Helvetica", 8)

    # Box outline
    c.setLineWidth(0.6)
    c.rect(ox, oy, w_mm * scale, h_mm * scale)

    df_code = df[df['Code'] == code].copy()

    def get_filtered_unique(vals, tol):
        filtered = []
        for v in sorted(vals):
            if not filtered or abs(v - filtered[-1]) > tol:
                filtered.append(v)
        return filtered

    tolerance = 0.5
    unique_xs = get_filtered_unique(df_code['X (mm)'].unique(), tolerance)
    unique_ys = get_filtered_unique(df_code['Y (mm)'].unique(), tolerance)

    # X dimension ticks
    c.setFont("Helvetica", 6)
    prev_x = None
    toggle_offset = False
    for x in unique_xs:
        dx = ox + x * scale
        if prev_x is not None and abs(x - prev_x) < 100:
            tick_len = 6 * mm
            text_offset = 7 * mm if toggle_offset else 3 * mm
            toggle_offset = not toggle_offset
        else:
            tick_len = 3 * mm
            text_offset = 4 * mm
            toggle_offset = False
        c.line(dx, oy, dx, oy - tick_len)
        c.drawCentredString(dx, oy - tick_len - text_offset, f"{x:.1f}")
        prev_x = x

    for y in unique_ys:
        dy = oy + y * scale
        c.line(ox, dy, ox - 2 * mm, dy)
        c.drawRightString(ox - 3 * mm, dy - 1 * mm, f"{y:.1f}")

    # Holes
    r = 0.6 * mm
    c.setLineWidth(0.5)
    for _, row in df_code.iterrows():
        dx = ox + row['X (mm)'] * scale
        dy = oy + row['Y (mm)'] * scale
        c.circle(dx, dy, r, stroke=1, fill=1)

# Add second page with table from df_holes
c.showPage()
c.setFont("Helvetica-Bold", 14)
c.drawCentredString(PAGE_W / 2, PAGE_H - 30 * mm, "Hole Table")

if 'df_holes' in globals():
    df_display = df_holes.copy()
    for col in df_display.select_dtypes(include=[float]).columns:
        df_display[col] = df_display[col].map(lambda x: f"{x:.1f}")

    headers = list(df_display.columns)
    col_width = (PAGE_W - 2 * MARGIN) / len(headers)
    row_height = 6 * mm
    x_start = MARGIN
    y_start = PAGE_H - 40 * mm

    # Draw header background and text
    c.setFont("Helvetica-Bold", 8)
    for i, header in enumerate(headers):
        x = x_start + i * col_width
        c.rect(x, y_start, col_width, row_height, stroke=1, fill=0)
        c.drawCentredString(x + col_width / 2, y_start + 1.5 * mm, str(header))

    # Draw table rows
    c.setFont("Helvetica", 8)
    for row_idx, row in df_display.iterrows():
        y = y_start - (row_idx + 1) * row_height
        for col_idx, header in enumerate(headers):
            x = x_start + col_idx * col_width
            c.rect(x, y, col_width, row_height, stroke=1, fill=0)
            c.drawCentredString(x + col_width / 2, y + 1.5 * mm, str(row[header]))

c.save()
print("Saved PDF: hole_drawing_views.pdf")


Saved PDF: hole_drawing_views.pdf
